In [ ]:
import cv2
import numpy as np
import os



In [2]:
# ========== STEP 1: Stabilize Video ==========
def stabilize_video(video_path):
    cap = cv2.VideoCapture(video_path)
    ret, prev = cap.read()
    if not ret:
        raise ValueError("Could not read the first frame.")

    prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    stabilized_frames = [prev]
    transforms = []

    while True:
        ret, curr = cap.read()
        if not ret:
            break

        curr_gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)
        prev_pts = cv2.goodFeaturesToTrack(prev_gray, maxCorners=200, qualityLevel=0.01, minDistance=30)
        curr_pts, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_pts, None)

        idx = np.where(status == 1)[0]
        prev_pts = prev_pts[idx]
        curr_pts = curr_pts[idx]

        m, _ = cv2.estimateAffinePartial2D(prev_pts, curr_pts)
        if m is None:
            m = np.eye(2, 3)

        stabilized = cv2.warpAffine(curr, m, (curr.shape[1], curr.shape[0]))
        stabilized_frames.append(stabilized)
        prev_gray = curr_gray.copy()

    cap.release()
    return stabilized_frames



In [3]:
# ========== STEP 2: Detect Motion Region ==========
def detect_motion_roi(frames, min_area=500):
    motion_accum = None

    for i in range(1, len(frames)):
        gray1 = cv2.cvtColor(frames[i - 1], cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)
        diff = cv2.absdiff(gray1, gray2)
        _, thresh = cv2.threshold(diff, 15, 255, cv2.THRESH_BINARY)

        if motion_accum is None:
            motion_accum = thresh
        else:
            motion_accum = cv2.bitwise_or(motion_accum, thresh)

    contours, _ = cv2.findContours(motion_accum, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    largest_box = None
    max_area = 0
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        area = w * h
        if area > max_area and area > min_area:
            largest_box = (x, y, w, h)
            max_area = area

    return largest_box



In [4]:
# ========== STEP 3: Save Preprocessed Frames ==========
def save_preprocessed_frames(frames, roi_box, output_dir, equalize_hist=True):
    os.makedirs(output_dir, exist_ok=True)
    for i, frame in enumerate(frames):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if roi_box:
            x, y, w, h = roi_box
            gray = gray[y:y + h, x:x + w]
        if equalize_hist:
            gray = cv2.equalizeHist(gray)
        cv2.imwrite(os.path.join(output_dir, f"frame_{i:04d}.png"), gray)



In [5]:
# ========== MAIN FUNCTION ==========
def preprocess_video_pipeline(video_path, output_dir):
    print("Stabilizing video...")
    frames = stabilize_video(video_path)

    print("Detecting vibrating region...")
    roi = detect_motion_roi(frames)

    print("Saving preprocessed frames...")
    save_preprocessed_frames(frames, roi, output_dir)

    print("✅ Done! Frames saved in:", output_dir)




In [ ]:
# ========== RUN IT ==========
if __name__ == "__main__":
    input_video = "input_data/Unbalanced_weight/front.mp4"
    output_folder = "preprocessed/unbalanced_weight_front"
    preprocess_video_pipeline(input_video, output_folder)